# Рекомендация тарифов

В вашем распоряжении данные о поведении клиентов, которые уже перешли на тарифы "Смарт" и "Ультра" (из проекта "Анализ прибыльности телефонных тарифов"). Необходимо построить модель для задачи классификации, которая выберет подходящий тариф. 

Построим модель с максимально большим значением *accuracy*. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверить *accuracy* на тестовой выборке самостоятельно.

## Откройте и изучите файл

In [1]:
# импортируем библиотеки и загрузим датасет
# рассмотрим первые 5 строк таблицы
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.dummy import DummyClassifier

import warnings
warnings.filterwarnings("ignore")

try:
    data=pd.read_csv('/datasets/users_behavior.csv')
except:
    data=pd.read_csv('https://code.s3.yandex.net/datasets/users_behavior.csv')

data.head()

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


In [2]:
#раасмотрим общую информацию о датасете
data.info()
data.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


,calls,minutes,messages,mb_used,is_ultra
count,3214.000000,3214.000000,3214.000000,3214.000000,3214.000000
mean,63.038892,438.208787,38.281269,17207.673836,0.306472
std,33.236368,234.569872,36.148326,7570.968246,0.461100
min,0.000000,0.000000,0.000000,0.000000,0.000000
25%,40.000000,274.575000,9.000000,12491.902500,0.000000
50%,62.000000,430.600000,30.000000,16943.235000,0.000000
75%,82.000000,571.927500,57.000000,21424.700000,1.000000
max,244.000000,1632.060000,224.000000,49745.730000,1.000000


In [3]:
# изменим формат данных
data['calls'] = data['calls'].astype(int)
data['messages'] = data['messages'].astype(int)
#data['is_ultra'] = data['is_ultra'].astype(bool)

In [4]:
#Проверим на наличие явных дубликатов
data.duplicated().sum()

0

Таблица содержит 3214 строк и 5 столбцов. Пропусков в данных не обнаруженно. Ниманования столбцов заданны корректно. Изменен формат данных в стобцах  calls и messages, т.к. колличество звонков и смс не может быть дробным числом.

## Разбейте данные на выборки

In [5]:
# извлекаем матрицу признаков
features = data.drop(['is_ultra'], axis=1) 
# извлекаем целевой признак
target = data['is_ultra']

Так как спрятанной тестовой выборки у нас нет. Данные нужно разбить на три части: обучающую, валидационную и тестовую. Размеры тестового и валидационного наборов обычно равны. Разобьем исходные данные в соотношении 3:1:1.(60%:20%:20%)

In [6]:
# отделим 40% данных для тестовой и валидационной выборок
features_train, features_test, target_train, target_test = train_test_split(
    features, target, test_size=0.40, random_state=12345)

In [7]:
# отделим 20% данных для валидационной выборки
features_test, features_valid, target_test, target_valid = train_test_split(
    features_test, target_test, test_size=0.50, random_state=12345) 

In [8]:
print('features_train','',features_train.shape[0])
print('features_test','',features_test.shape[0])
print('features_valid','',features_valid.shape[0])

features_train  1928
features_test  643
features_valid  643


Таким образом все данные были разделены на 3 группы:
- features_train, target_train - обучающая выборка (60%)
- features_valid, target_valid - валидационная выборка (20%)
- features_test, target_test - тестовая выборка (20%)

## Проверьте модель на тестовой выборке

Используем модели для задач классификациии

### Дерево решений 

In [9]:
best_accuracy = 0
best_depth = 0
best_sample = 0 

for depth in range(1,10):
    for sample in range(2,5):
        model = DecisionTreeClassifier(max_depth=depth, random_state=321,min_samples_split =sample)
        model.fit(features_train, target_train)
        accuracy = model.score(features_valid, target_valid)
        if accuracy > best_accuracy:
            best_accuracy = accuracy
            best_depth = depth
            best_sample = sample
print("Глубина дерева:", best_depth, "\nОбъектов в узле:",best_sample, "\nAccuracy лучшей модели:", best_accuracy)

Глубина дерева: 8 
Объектов в узле: 2 
Accuracy лучшей модели: 0.7962674961119751


### Случайный лес 

In [10]:
best_accuracy = 0
best_depth = 0
for depth in range(1,8):
    for est in range(10,30): 
        for sample in range(2,5):
            model = RandomForestClassifier(criterion='gini', max_depth=depth, n_estimators=est, 
                                           min_samples_leaf=sample, random_state=321)
            model.fit(features_train, target_train)
            accuracy = model.score(features_valid, target_valid)
            if accuracy > best_accuracy:
                best_accuracy = accuracy
                best_depth = depth
                best_est = est
                best_sample = sample
print("Глубина дерева:", best_depth, 
      "\nКоличество деревьев:",best_est, 
      "\nОбъектов в узле:", best_sample, 
      "\nAccuracy лучшей модели:", best_accuracy)

Глубина дерева: 7 
Количество деревьев: 19 
Объектов в узле: 3 
Accuracy лучшей модели: 0.80248833592535


### Логическая регрессия

In [11]:
model = LogisticRegression(random_state=321, solver='lbfgs', max_iter=1000)
model.fit(features_train, target_train)
accuracy = model.score(features_valid, target_valid)
print("Accuracy лучшей модели:", accuracy)

Accuracy лучшей модели: 0.6842923794712286


Основываясь на показателе точности (Accuracy), можно сделать вывод, что наиболее качественные результаты прогнозирует Случайный лес (Random Forest)

### Проверим модель на тестовой выборке

In [12]:
#Дерево решений
model = DecisionTreeClassifier(max_depth=8, random_state=321,min_samples_split =2)
model.fit(features_train, target_train)
accuracy_t = model.score(features_test, target_test)
print("Accuracy:", accuracy_t)

Accuracy: 0.7822706065318819


In [13]:
#Случайный лес
model = RandomForestClassifier(criterion='gini', max_depth=7, n_estimators=19, min_samples_leaf=3, random_state=321)
model.fit(features_train, target_train)
accuracy_f = model.score(features_test, target_test)
print("Accuracy:", accuracy_f)

Accuracy: 0.7916018662519441


In [14]:
#Логическая регрессия
model = LogisticRegression(random_state=321, solver='lbfgs', max_iter=1000)
model.fit(features_train, target_train)
accuracy_r = model.score(features_test, target_test)
print("Accuracy:", accuracy_r)

Accuracy: 0.7107309486780715


Были проверены три модели:дерево решений,случайный лес и логистическая регрессия.
Наиболее точные предсказания делает Случайный лес (Random Forest), как на валидационной, так и на тестовой выборке.
Наилучшее качество (0.79) достигается при использовании RandomForestClassifier с гиперпараметрами max_depth=9 , n_estimators=43 , min_samples_leaf=3.

## (бонус) Проверьте модели на адекватность

Для проверки используем модель DummyClassifier. Для праметра strategy зададим знаечние uniform, т.к. оно генерирует предсказания равномерно случайным образом из списка уникальных классов.

In [15]:
model_d = DummyClassifier(strategy='uniform', random_state=321)
model_d.fit(features_train, target_train)
result_d = model_d.score(features_valid, target_valid)
print('Accuracy DummyClassifier:', result_d)
print("Accuracy модели Случайный лес:", accuracy_f)

Accuracy DummyClassifier: 0.48833592534992226
Accuracy модели Случайный лес: 0.7916018662519441


Accuracy модели DummyClassifier значительно ниже, чем у модели Случайного леса. Из чего можно сделать вывод, что выбранная модель адекватна.